In [ ]:
# default_exp bridge

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Bridge

In [ ]:
#export
from bfh_mt_hs2020_rl_basics.agent import Agent

from typing import Iterable, Tuple, List
import numpy as np

from ignite.engine import Engine

from ptan.experience import ExperienceFirstLast

import torch
import torch.nn as nn
from torch import device

class Bridge:
    
    def __init__(self, agent: Agent, device: device, gamma: float, target_net_sync: int):
        self.gamma = gamma
        self.target_net_sync = target_net_sync
        
        self.device = device    
        self.agent = agent

        
    def process_batch(self, engine:Engine, batch: List[ExperienceFirstLast]):
        optimizer.zero_grad()
        loss_v = self._calc_loss(batch)

        loss_v.backward()
        optimizer.step()
        epsilon_tracker.frame(engine.state.iteration)
        
        if engine.state.iteration % self.target_net_sync == 0:
            self.tgt_net.sync()

        return {
            "loss": loss_v.item(),
            "epsilon": selector.epsilon,
        }


    def _calc_loss(self, batch: List[ExperienceFirstLast]):
        
        states, actions, rewards, dones, next_states = self._unpack_batch(batch)

        states_v      = torch.tensor(states).to(self.device)
        next_states_v = torch.tensor(next_states).to(self.device)
        actions_v     = torch.tensor(actions).to(self.device)
        rewards_v     = torch.tensor(rewards).to(self.device)
        done_mask     = torch.BoolTensor(dones).to(self.device)

        actions_v         = actions_v.unsqueeze(-1)
        state_action_vals = self.agent.net(states_v).gather(1, actions_v)
        state_action_vals = state_action_vals.squeeze(-1)
        
        with torch.no_grad():
            next_state_vals            = self.agent.tgt_net(next_states_v).max(1)[0]
            next_state_vals[done_mask] = 0.0
    
        bellman_vals = next_state_vals.detach() * self.gamma + rewards_v
        return nn.MSELoss()(state_action_vals, bellman_vals)


    def _unpack_batch(self, batch: List[ExperienceFirstLast]):
        states, actions, rewards, dones, last_states = [],[],[],[],[]
        
        for exp in batch:
            state = np.array(exp.state)
            states.append(state)
            actions.append(exp.action)
            rewards.append(exp.reward)
            dones.append(exp.last_state is None)
            
            if exp.last_state is None:
                lstate = state  # the result will be masked anyway
            else:
                lstate = np.array(exp.last_state)
            last_states.append(lstate)
            
        return np.array(states, copy=False), \
               np.array(actions), \
               np.array(rewards, dtype=np.float32), \
               np.array(dones,   dtype=np.uint8), \
               np.array(last_states, copy=False)

In [ ]:
from bfh_mt_hs2020_rl_basics.agent import Agent
from bfh_mt_hs2020_rl_basics.env import CarEnv

def basic_init() -> Bridge:
    env = CarEnv()
    agent = Agent(env, 0.9, 1000)
    bridge = Bridge(agent, torch.device("cpu"), 0.9, 100)
    
    return bridge

In [ ]:
def simple_experiences() -> List[ExperienceFirstLast]:
    return [
        ExperienceFirstLast([], 0, 1.0, []),
        ExperienceFirstLast(3, 1, 2.0, 5)        
    ]

In [ ]:
def test_init():
    assert basic_init() != None

In [ ]:
def test_unpack():
    bridge = basic_init()
    batch = simple_experiences()
    unpacked = bridge._unpack_batch(batch)
    # todo -Checks

In [ ]:
def test_calc_loss():
    bridge = basic_init()
    batch = simple_experiences()
    loss = bridge._calc_loss(batch)
    # todo -Checks  

In [ ]:
test_unpack()

In [ ]:
test_calc_loss()

RuntimeError: size mismatch, m1: [1 x 2], m2: [3 x 128] at ..\aten\src\TH/generic/THTensorMath.cpp:41

In [ ]:
test_init()